# full fine tuning

In [1]:
%matplotlib inline

In [2]:
!pip install Pillow
!pip install scipy

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import numpy as np
import scipy 
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import IPython.display as display
import time

keras = tf.keras
print(tf.__version__)

2.0.0


## Get tha data

In [4]:
data_dir = './dataset/trigo/croped/'
data_dir = pathlib.Path(data_dir)

In [5]:
image_count = len(list(data_dir.glob('*/*.jpg'))) + len(list(data_dir.glob('*/*.png')))
image_count

840

In [6]:
N = 0  # total files
for dirpath, dirnames, filenames in os.walk(data_dir):    
    dirpath = dirpath.split("/")[-1]
    if dirpath != ".ipynb_checkpoints":
        N_c = len(filenames)
        N += N_c
        print( dirpath+ ": -> " + str(N_c))
print( "Total Files " + str(N) )

croped: -> 0
roya: -> 265
mancha_foliar: -> 189
oidio: -> 194
sano: -> 192
Total Files 840


In [7]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != ".ipynb_checkpoints"])
CLASS_NAMES

array(['roya', 'mancha_foliar', 'oidio', 'sano'], dtype='<U13')

In [8]:
# The 1./255 is to convert from uint8 to float32 in range [0,1].
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  validation_split=0.15,                            
                                                                  zoom_range=0.4,
                                                                  horizontal_flip=True,
                                                                  rotation_range=90
                                                                 )

In [15]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
STEPS_PER_EPOCH = np.ceil((image_count*0.85)/BATCH_SIZE)
int(STEPS_PER_EPOCH)

23

In [16]:
train_data_gen = image_generator.flow_from_directory(directory= str(data_dir),
                                                     batch_size= BATCH_SIZE,
                                                     shuffle= True,
                                                     target_size= (IMG_HEIGHT, IMG_WIDTH),
                                                     subset = "training",
                                                     classes = list(CLASS_NAMES))

Found 716 images belonging to 4 classes.


In [17]:
valid_data_gen = image_generator.flow_from_directory(directory= str(data_dir),
                                                     batch_size= BATCH_SIZE,
                                                     shuffle= True,
                                                     target_size= (IMG_HEIGHT, IMG_WIDTH),
                                                     subset = "validation",
                                                     classes = list(CLASS_NAMES))

Found 124 images belonging to 4 classes.


## Get the model

In [38]:
base_model = tf.keras.applications.xception.Xception(include_top=False, 
                                                     weights='imagenet', 
                                                     input_shape=IMG_SHAPE
                                                    )

In [39]:
base_model.trainable = False

In [40]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")

In [41]:
model_x = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [42]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.4,
                              patience=2, min_lr=0.0001, verbose = 1, min_delta = 0.005)
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6 ,restore_best_weights=True)
filepath_x = "models/imagenet_trigo_Xception.{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = filepath_x, 
                                                          save_best_only=True , monitor='val_accuracy', mode='max')

In [43]:
base_learning_rate = 0.03
model_x.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [44]:
model_x.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 8196      
Total params: 20,869,676
Trainable params: 8,196
Non-trainable params: 20,861,480
_________________________________________________________________


In [45]:
total_epochs = 150
start_time = time.time()
history_x = model_x.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                         callbacks = [checkpoint]
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

Epoch 1/150
23/23 [==============================] - 61s 3s/step - loss: 2.2681 - accuracy: 0.4078 - val_loss: 3.1327 - val_accuracy: 0.5323
Epoch 2/150
23/23 [==============================] - 53s 2s/step - loss: 1.3818 - accuracy: 0.5628 - val_loss: 3.1617 - val_accuracy: 0.3952
Epoch 3/150
23/23 [==============================] - 54s 2s/step - loss: 1.2406 - accuracy: 0.6285 - val_loss: 3.6841 - val_accuracy: 0.3952
Epoch 4/150
23/23 [==============================] - 54s 2s/step - loss: 1.0918 - accuracy: 0.6480 - val_loss: 2.7631 - val_accuracy: 0.3871
Epoch 5/150
23/23 [==============================] - 54s 2s/step - loss: 1.0838 - accuracy: 0.6536 - val_loss: 2.6320 - val_accuracy: 0.5081
Epoch 6/150
23/23 [==============================] - 54s 2s/step - loss: 0.8720 - accuracy: 0.6732 - val_loss: 2.1896 - val_accuracy: 0.5000
Epoch 7/150
23/23 [==============================] - 54s 2s/step - loss: 0.7856 - accuracy: 0.6983 - val_loss: 2.5356 - val_accuracy: 0.4919
Epoch 8/150
2

Epoch 59/150
23/23 [==============================] - 54s 2s/step - loss: 1.1466 - accuracy: 0.7025 - val_loss: 3.7750 - val_accuracy: 0.4677
Epoch 60/150
23/23 [==============================] - 52s 2s/step - loss: 1.0345 - accuracy: 0.7332 - val_loss: 4.5468 - val_accuracy: 0.4435
Epoch 61/150
23/23 [==============================] - 53s 2s/step - loss: 1.0176 - accuracy: 0.7346 - val_loss: 3.2087 - val_accuracy: 0.4435
Epoch 62/150
23/23 [==============================] - 52s 2s/step - loss: 0.9803 - accuracy: 0.7374 - val_loss: 4.2997 - val_accuracy: 0.4355
Epoch 63/150
23/23 [==============================] - 53s 2s/step - loss: 0.9014 - accuracy: 0.7542 - val_loss: 4.7605 - val_accuracy: 0.3790
Epoch 64/150
23/23 [==============================] - 52s 2s/step - loss: 1.1648 - accuracy: 0.7095 - val_loss: 4.9627 - val_accuracy: 0.4113
Epoch 65/150
23/23 [==============================] - 53s 2s/step - loss: 1.4784 - accuracy: 0.6830 - val_loss: 3.3545 - val_accuracy: 0.5081
Epoch 

23/23 [==============================] - 53s 2s/step - loss: 0.9077 - accuracy: 0.7584 - val_loss: 3.6936 - val_accuracy: 0.5403
Epoch 117/150
23/23 [==============================] - 53s 2s/step - loss: 0.8786 - accuracy: 0.7556 - val_loss: 4.3934 - val_accuracy: 0.4597
Epoch 118/150
23/23 [==============================] - 53s 2s/step - loss: 1.1375 - accuracy: 0.7095 - val_loss: 5.7465 - val_accuracy: 0.3952
Epoch 119/150
23/23 [==============================] - 53s 2s/step - loss: 1.3138 - accuracy: 0.6746 - val_loss: 4.2670 - val_accuracy: 0.4597
Epoch 120/150
23/23 [==============================] - 52s 2s/step - loss: 1.1343 - accuracy: 0.7291 - val_loss: 4.8759 - val_accuracy: 0.4355
Epoch 121/150
23/23 [==============================] - 54s 2s/step - loss: 1.0770 - accuracy: 0.7472 - val_loss: 5.4132 - val_accuracy: 0.4274
Epoch 122/150
23/23 [==============================] - 53s 2s/step - loss: 0.9967 - accuracy: 0.7388 - val_loss: 5.5307 - val_accuracy: 0.4435
Epoch 123/150